In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb

In [3]:
import pickle

with open("/home/bruce1996/nvme2/scRNA/GSE149614_overall_normalized.pkl",'rb') as f :
    scRNA_exp_m = pickle.load(f)
f.close()

with open("/home/bruce1996/data/LIHC_anomaly_detection/data/coding_gene_info/ensembl2hgnc.pkl",'rb') as f :
    tmp = pickle.load(f)
f.close()
hgnc2ens = {}
for ens,hgnc in tmp.items() :
    if isinstance(hgnc,str) :
        hgnc2ens[hgnc] = ens
ens2hgnc = {v:key for key,v in hgnc2ens.items()}
metadata = pd.read_csv('/home/bruce1996/data/LIHC_anomaly_detection/validation_dataset/scRNA/GSE149614_HCC.metadata.txt',sep='\t',index_col=0)

In [ ]:
idx = metadata.index[np.where((metadata['Celltype'] == 'Hepatocyte') & (metadata['site'] == 'Tumor'),True,False)]
exp_m = scRNA_exp_m.loc[:,idx]
metadata = metadata.loc[idx,:]

del scRNA_exp_m
coding_gene = set(exp_m.index).intersection(hgnc2ens.keys())
coding_m = exp_m.loc[coding_gene,:]
coding_m.index = [hgnc2ens[x] for x in coding_m.index]

In [ ]:
with open("/home/bruce1996/nvme2/scRNA/GSE149614_coding_gene_hepatocyte_normalized.pkl",'wb') as f :
    pickle.dump(coding_m,f)
f.close()

vote = pd.read_csv("/home/bruce1996/data/LIHC_anomaly_detection/manuscript/material/ensemble_learning_result/ensemble_hbv_only_np_ratio_35_vote_result.txt",sep='\t',index_col=0)
voted_gene = list(set(vote.index[vote['Vote'] > 0]).intersection(coding_m.index))
voted_exp_m = coding_m.loc[voted_gene,:]